In [34]:
# _*_ coding: utf-8 _*_
# @Time: 2021/11/29 24:17 
# @Author: yuyongsheng
# @Software: PyCharm
# @Description: 上海六院甲氨蝶呤：3. 特征工程

In [35]:
# 导入程序包
import pymysql as MySQLDB
import pandas as pd
import numpy as np

import re
import sys
import os
project_path = os.getcwd()

# 特征清洗

In [82]:
df_model=pd.read_excel(project_path+'/data/processed_data/df_11.4_人工删除无意义列.xlsx')
if 'Unnamed: 0' in df_model.columns:
    df_model = df_model.drop(['Unnamed: 0'], axis=1)

## 数据转换

In [83]:
df_model['gender'] = df_model['gender'].str.replace(' ','').apply(lambda x: 1 if x == '男' else 0)

## 删除无关特征

In [84]:
df_model.columns

Index(['patient_id', 'case_no', 'drug_name', '日剂量', 'start_datetime',
       'end_datetime', 'gender', 'age', '血红蛋白_before', '血红蛋白_3d',
       ...
       '白蛋白(干式)', '碱性磷酸酶(干式)', '红细胞', '红细胞分布宽度', '肌酐(干式)', '血清氯(干式)',
       '血清钠(干式)', '血清钾(干式)', '谷丙转氨酶(干式)', '谷草转氨酶(干式)'],
      dtype='object', length=103)

In [85]:
# 删除无关特征
df_model=df_model.drop(['patient_id','case_no','drug_name','start_datetime','end_datetime','group','bmd_time'],axis=1)

In [86]:
# 删除目标变强强相关列
del df_model['bmd_before']
del df_model['bmd_3d']
del df_model['bmd_7d']
del df_model['bmd_14d']

df_model=df_model.drop(['血红蛋白_before','血红蛋白_3d','白细胞_before','白细胞_3d','粒细胞_before','血小板_before','血小板_3d',
                '血红蛋白_before_B','血红蛋白_3d_B','白细胞_before_B','白细胞_3d_B','粒细胞_before_B','血小板_before_B','血小板_3d_B'],axis=1)

## 删除缺失超过50%的特征

In [87]:
# 删除缺失超过50%的其他指标
for i in df_model.columns:
    if df_model[i].isnull().sum()/df_model.shape[0] >= 0.5:
        print(i)
        del df_model[i]

In [88]:
print(df_model.shape)  # (106,101)

(718, 78)


In [89]:
# 删除缺失超过50%的特征
writer=pd.ExcelWriter(project_path+'/data/result/feature_engineering/df_1.3_删除缺失超过50%.xlsx')
df_model.to_excel(writer)
writer.save()

## 删除分类极不平衡的特征

In [90]:
# 保护目标变量、tdm数据
for i in df_model.columns[8:]:
    if df_model[i].nunique() < 2:
        del df_model[i]
        continue
    if df_model[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_model[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_model.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_model[i]

饮酒史
高血压
糖尿病
肝炎
异环磷酰胺
顺铂
头孢类
促红素
抗凝血
地塞米松磷酸钠
甲氨蝶呤+顺铂
甲氨蝶呤+异环磷酰胺


In [91]:
print(df_model.shape) 

(718, 41)


In [92]:
# 删除缺失超过50%的特征
writer=pd.ExcelWriter(project_path+'/data/result/feature_engineering/df_1.4_删除分类极不平衡.xlsx')
df_model.to_excel(writer)
writer.save()

## 过滤异常值

In [93]:
# 过滤异常值，大于正常值超过100倍！
for i in df_model.columns[8:]:
    # 测试特征是否存在异常值
    median_value= max(df_model[i].median(),1)
    max_value = df_model[i].max()
    if max_value > 100 * median_value:
        print(i)
    df_model[i]=df_model[i].apply(lambda x: np.nan if x<0 or x!=x else 100*median_value if float(x) > 100*median_value else x)

红细胞


In [94]:
print(df_model.shape)  

(718, 41)


In [95]:
df_model

,日剂量,gender,age,bmd_label,MTX_tdm_24h,MTX_tdm_48h,MTX_tdm_72h,吸烟史,谷胱甘肽,异甘草酸镁,...,白蛋白(干式),碱性磷酸酶(干式),红细胞,红细胞分布宽度,肌酐(干式),血清氯(干式),血清钠(干式),血清钾(干式),谷丙转氨酶(干式),谷草转氨酶(干式)
0,20.0,1,45,0,28.32,3.48,3.44,0.0,1,1,...,36.0,91.0,3.78,15.0,62.0,101.0,142.0,3.5,81.0,45.0
1,11.0,0,9,0,6.90,0.10,NaN,0.0,1,1,...,40.0,159.0,4.11,14.2,35.0,102.0,142.0,4.4,173.0,46.0
2,21.0,1,24,1,14.83,4.85,NaN,0.0,0,0,...,38.0,59.0,0.00,14.2,57.0,103.0,143.0,3.8,86.0,69.0
3,13.0,0,15,1,9.21,3.79,1.34,0.0,0,0,...,38.0,77.0,3.23,15.6,39.0,100.0,139.0,3.8,353.0,86.0
4,16.0,1,20,0,57.57,3.74,0.10,0.0,1,0,...,40.0,64.0,3.05,17.1,61.0,102.0,141.0,4.3,113.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,18.0,1,23,0,0.90,0.10,0.10,0.0,0,1,...,37.0,119.0,3.58,12.3,49.0,100.0,139.0,4.7,34.0,18.0
714,16.5,1,18,1,14.30,1.13,0.63,0.0,0,1,...,37.0,85.0,4.28,21.4,66.0,106.0,142.0,4.1,47.0,16.0
715,12.4,1,12,0,5.66,0.10,0.10,0.0,1,1,...,41.0,148.0,2.00,16.0,42.0,103.0,144.0,3.5,147.0,83.0
716,16.0,0,37,0,0.10,NaN,NaN,0.0,0,0,...,27.0,57.0,2.85,16.6,33.0,99.0,138.0,4.0,25.0,20.0


In [96]:
# 保存特征清洗
writer = pd.ExcelWriter(project_path + '/data/result/feature_engineering/df_1_特征清洗.xlsx')
df_model.to_excel(writer)
writer.save()

# 显著性分析

In [97]:
discrete_col=['gender','谷胱甘肽','异甘草酸镁','托烷司琼','甘草酸苷','长春新碱','多烯磷脂酰']
continuous_col=[x for x in df_model.columns if x not in discrete_col]
continuous_col.remove('bmd_label')

## 分类变量显著性分析

In [98]:
# 二分类卡方检验
from scipy.stats import chi2_contingency
feature_list=[]
y_list=[]
t_list=[]
p_list=[]
sig_list=[]
for i in discrete_col:
    print(i)
    result = chi2_contingency(pd.crosstab(df_model['bmd_label'],df_model[i]))
    t,p=result[0:2]
    feature_list.append(i)
    y_list.append('卡方检验')
    t_list.append(t)
    p_list.append(p)
    if p <=0.05:
        sig='显著'
    else:
        sig='不显著'
    sig_list.append(sig)

gender
谷胱甘肽
异甘草酸镁
托烷司琼
甘草酸苷
长春新碱
多烯磷脂酰


In [99]:
df_result_dis=pd.DataFrame({'特征':feature_list,
                        '检验指标':y_list,
                        't值':t_list,
                        'p值':p_list,
                        '显著性结果':sig_list})

In [100]:
# psm分类变量显著性检验
writer=pd.ExcelWriter(project_path+'/data/result/feature_engineering/df_2.1_分类变量显著性检验.xlsx')
df_result_dis.to_excel(writer)
writer.save()

## 连续变量显著性分析

In [101]:
from scipy.stats import kstest,shapiro
##检验是否正态
def norm_test(data):
    if len(data) > 30:
        norm, p = kstest(data, 'norm')
    else:
        norm, p = shapiro(data)
    #print(t,p)
    if p>=0.05:
        return True
    else:
        return False

In [102]:
import scipy.stats as st
# 连续变量的显著性检验
def test2(data_b, data_p):
    if norm_test(data_b) and norm_test(data_p):
        x = 1
        y = '独立样本T检验'
        t, p = st.ttest_ind(list(data_b),list(data_p), nan_policy='omit')
    else:
        x = 0
        y = 'Mann-Whitney U检验'
        t,p = st.mannwhitneyu(list(data_b),list(data_p))
    return x,y,t,p

In [103]:
def sig_test(df_high,df_low,list1):

    feature_list=[]  # 特征列表
    y_list=[]  # 显著性检验方法
    t_list=[]  # 统计量
    p_list=[]  # p值
    result_list=[]  # 是否显著
    high_mean_list=[]
    low_mean_list=[]

    for i in list1:
        print(i)
        # 高剂量组统计
        df_high_nt=df_high[df_high[i].notnull()]
        data_high=df_high_nt[i]
        high_mean=round(data_high.mean(),2)
        
        df_low_nt=df_low[df_low[i].notnull()]
        data_low=df_low_nt[i]
        low_mean=round(data_low.mean(),2)

        # 计算高低剂量组显著性差异
        if data_high.shape[0] >= 10 and data_low.shape[0]>=10:
            # 连续变量检验
            x,y,t,p = test2(data_high, data_low)
            if p <=0.05:
                sig='显著'
            else:
                sig='不显著'
            # 显著性 
            feature_list.append(i)
            y_list.append(y)
            t_list.append(t)
            p_list.append(p)
            result_list.append(sig)
            high_mean_list.append(high_mean)
            low_mean_list.append(low_mean)

    df_result=pd.DataFrame({'特征':feature_list,
                            '高剂量均值':high_mean_list,
                            '低剂量均值':low_mean_list,
                            '检验指标':y_list,
                            't值':t_list,
                            'p值':p_list,
                            '显著性结果':result_list})
    return df_result

In [104]:
# 提取甲氨蝶呤不同标签
df_MTX_high = df_model[df_model['bmd_label']==1]
df_MTX_low = df_model[df_model['bmd_label']==0]

In [105]:
# 甲氨蝶呤高低剂量组PSM控制变量统计和显著性检验
df_MTX_PSM_sig = sig_test(df_MTX_high,df_MTX_low,continuous_col)
df_MTX_PSM_sig = df_MTX_PSM_sig.reset_index(drop=True)

日剂量
age
MTX_tdm_24h
MTX_tdm_48h
MTX_tdm_72h
吸烟史
RBC平均容量
RBC平均血红量
RBC血红浓度
γ-谷氨酰酶(干式)
中性细胞百分比
中性细胞绝对值
乳酸脱氢酶(干式)
单核细胞百分比
单核细胞绝对值
嗜碱性细胞百分比
嗜碱性细胞绝对值
嗜酸性细胞百分比
嗜酸性细胞绝对值
总胆红素(干式)
总蛋白(干式)
淋巴细胞百分比
淋巴细胞绝对值
白蛋白(干式)
碱性磷酸酶(干式)
红细胞
红细胞分布宽度
肌酐(干式)
血清氯(干式)
血清钠(干式)
血清钾(干式)
谷丙转氨酶(干式)
谷草转氨酶(干式)


In [106]:
# 保存高低剂量组PSM控制变量统计和显著性检验
writer=pd.ExcelWriter(project_path+r'/data/result/feature_engineering/df_2.2_连续变量相关性分析.xlsx')
df_MTX_PSM_sig.to_excel(writer)
writer.save()

In [110]:
continuous_relate = list(df_MTX_PSM_sig[df_MTX_PSM_sig['显著性结果']=='不显著']['特征'])

In [111]:
# 人工调整
continuous_relate.extend(['红细胞','肌酐(干式)','谷草转氨酶(干式)'])

In [112]:
continuous_relate

['日剂量',
 'age',
 'MTX_tdm_24h',
 'MTX_tdm_48h',
 'MTX_tdm_72h',
 '吸烟史',
 'RBC平均容量',
 'RBC平均血红量',
 'RBC血红浓度',
 'γ-谷氨酰酶(干式)',
 '嗜碱性细胞绝对值',
 '总胆红素(干式)',
 '碱性磷酸酶(干式)',
 '红细胞分布宽度',
 '血清钠(干式)',
 '血清钾(干式)',
 '谷丙转氨酶(干式)',
 '红细胞',
 '肌酐(干式)',
 '谷草转氨酶(干式)']

# 重要性评分排序

In [113]:
# 提取相关性特征
feature_relate=discrete_col + continuous_relate
# 加入目标变量
feature_relate.insert(0,'bmd_label')

In [115]:
df_model_relate=df_model[feature_relate]

In [116]:
df_model_relate

,bmd_label,gender,谷胱甘肽,异甘草酸镁,托烷司琼,甘草酸苷,长春新碱,多烯磷脂酰,日剂量,age,...,嗜碱性细胞绝对值,总胆红素(干式),碱性磷酸酶(干式),红细胞分布宽度,血清钠(干式),血清钾(干式),谷丙转氨酶(干式),红细胞,肌酐(干式),谷草转氨酶(干式)
0,0,1,1,1,1,0,0,0,20.0,45,...,0.04,9.0,91.0,15.0,142.0,3.5,81.0,3.78,62.0,45.0
1,0,0,1,1,0,0,0,0,11.0,9,...,0.01,12.0,159.0,14.2,142.0,4.4,173.0,4.11,35.0,46.0
2,1,1,0,0,0,0,0,0,21.0,24,...,0.02,21.0,59.0,14.2,143.0,3.8,86.0,0.00,57.0,69.0
3,1,0,0,0,0,0,0,0,13.0,15,...,0.01,8.0,77.0,15.6,139.0,3.8,353.0,3.23,39.0,86.0
4,0,1,1,0,0,1,0,0,16.0,20,...,0.01,7.0,64.0,17.1,141.0,4.3,113.0,3.05,61.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,0,1,0,1,0,0,0,0,18.0,23,...,0.00,11.0,119.0,12.3,139.0,4.7,34.0,3.58,49.0,18.0
714,1,1,0,1,1,0,0,0,16.5,18,...,0.02,8.0,85.0,21.4,142.0,4.1,47.0,4.28,66.0,16.0
715,0,1,1,1,1,0,0,0,12.4,12,...,0.02,9.0,148.0,16.0,144.0,3.5,147.0,2.00,42.0,83.0
716,0,0,0,0,0,0,0,0,16.0,37,...,0.00,7.0,57.0,16.6,138.0,4.0,25.0,2.85,33.0,20.0


In [121]:
from sklearn.model_selection import train_test_split

# 划分训练集和测试集，比例为8:2
x = df_model_relate.drop(['bmd_label'],axis=1)
y = df_model_relate['bmd_label']
tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=5)

In [122]:
# 重要性
import catboost,xgboost
model_boost=xgboost.XGBClassifier(max_depth=5,
                                learning_rate=0.01,
                                n_estimators=500,
                                min_child_weight=0.5,
                                eta=0.1,
                                gamma=0.5,
                                reg_lambda=10,
                                subsample=0.5,
                                colsample_bytree=0.8,
                                nthread=4,
                                scale_pos_weight=1)
model_boost.fit(tran_x,tran_y)
importance = model_boost.feature_importances_
print(tran_x.columns)
print(importance)

Index(['gender', '谷胱甘肽', '异甘草酸镁', '托烷司琼', '甘草酸苷', '长春新碱', '多烯磷脂酰', '日剂量',
       'age', 'MTX_tdm_24h', 'MTX_tdm_48h', 'MTX_tdm_72h', '吸烟史', 'RBC平均容量',
       'RBC平均血红量', 'RBC血红浓度', 'γ-谷氨酰酶(干式)', '嗜碱性细胞绝对值', '总胆红素(干式)',
       '碱性磷酸酶(干式)', '红细胞分布宽度', '血清钠(干式)', '血清钾(干式)', '谷丙转氨酶(干式)', '红细胞',
       '肌酐(干式)', '谷草转氨酶(干式)'],
      dtype='object')
[0.03570298 0.04878953 0.07653944 0.06042998 0.0685257  0.02581139
 0.03223845 0.02360842 0.0360296  0.02608755 0.025063   0.04174305
 0.02051579 0.03265351 0.02967918 0.02414707 0.02526583 0.02526722
 0.03455394 0.0374547  0.03799362 0.02067776 0.02644557 0.04605425
 0.0397789  0.02562527 0.07331826]


In [123]:
df_importance= pd.DataFrame(data={'特征':tran_x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)

In [124]:
writer = pd.ExcelWriter(project_path + '/data/result/feature_engineering/df_3_模型重要性评分.xlsx')
df_importance.to_excel(writer)
writer.save()

In [125]:
# 筛选重要特征组建建模数据
df_importance_select=df_importance[df_importance['重要性评分']>0]
select_list = list(df_importance_select['特征'])
select_list.insert(0,'bmd_label')

In [126]:
df_model_importance = df_model_relate[select_list]

In [127]:
df_model_importance

,bmd_label,异甘草酸镁,谷草转氨酶(干式),甘草酸苷,托烷司琼,谷胱甘肽,谷丙转氨酶(干式),MTX_tdm_72h,红细胞,红细胞分布宽度,...,血清钾(干式),长春新碱,肌酐(干式),MTX_tdm_48h,γ-谷氨酰酶(干式),嗜碱性细胞绝对值,RBC血红浓度,日剂量,吸烟史,血清钠(干式)
0,0,1,45.0,0,1,1,81.0,3.44,3.78,15.0,...,3.5,0,62.0,3.48,66.0,0.04,334.0,20.0,0.0,142.0
1,0,1,46.0,0,0,1,173.0,NaN,4.11,14.2,...,4.4,0,35.0,0.10,52.0,0.01,356.0,11.0,0.0,142.0
2,1,0,69.0,0,0,0,86.0,NaN,0.00,14.2,...,3.8,0,57.0,4.85,57.0,0.02,339.0,21.0,0.0,143.0
3,1,0,86.0,0,0,0,353.0,1.34,3.23,15.6,...,3.8,0,39.0,3.79,64.0,0.01,338.0,13.0,0.0,139.0
4,0,0,33.0,1,0,1,113.0,0.10,3.05,17.1,...,4.3,0,61.0,3.74,55.0,0.01,345.0,16.0,0.0,141.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,0,1,18.0,0,0,0,34.0,0.10,3.58,12.3,...,4.7,0,49.0,0.10,37.0,0.00,341.0,18.0,0.0,139.0
714,1,1,16.0,0,1,0,47.0,0.63,4.28,21.4,...,4.1,0,66.0,1.13,50.0,0.02,331.0,16.5,0.0,142.0
715,0,1,83.0,0,1,1,147.0,0.10,2.00,16.0,...,3.5,0,42.0,0.10,77.0,0.02,345.0,12.4,0.0,144.0
716,0,0,20.0,0,0,0,25.0,NaN,2.85,16.6,...,4.0,0,33.0,NaN,31.0,0.00,326.0,16.0,0.0,138.0


In [128]:
# 保存重要性评分筛选后的建模数据
writer = pd.ExcelWriter(project_path + '/data/result/df_model_data.xlsx')
df_model_importance.to_excel(writer)
writer.save()

# 逐步向前

In [ ]:
import itertools

import numpy as np
import pandas as pd

import os
project_path=os.getcwd()

import statsmodels.api as sm
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error,mean_absolute_error  # 均方误差
import warnings
warnings.filterwarnings("ignore")
import catboost,xgboost

In [ ]:
# 判断文件路径是否存在，如果不存在则创建该路径
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:  # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  # makedirs 创建文件时如果路径不存在会创建这个路径

In [ ]:
# 划分数据集
#     discrete_list=['gender','糖皮质激素','质子泵','钙离子阻抗剂','其他免疫抑制剂','克拉霉素或阿奇霉素']
#     continuous_list = [x for x in df_model.columns if x not in discrete_list]
# continuous_list.remove('TDM检测结果')
# 划分训练集和测试集，比例为8:2
x = df_model.drop(['bmd_label'],axis=1)
y = df_model['bmd_label']
#     # 连续变量归一化
#     for col in continuous_list:
#         df_model[col] = df_model[col].apply(lambda x: np.log(x) if x > 0 else np.nan if x != x else 0)
tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=5)

In [ ]:
tran_x

In [ ]:
tran_y

In [ ]:
# 逐步向前算法
from sklearn.preprocessing import StandardScaler
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import xgboost as xgb
import matplotlib.pyplot as plt

In [ ]:
# k_features,Number of features to select,where k_features < the full feature set.
# 基于当前所有变量中做逐步向前，从k=1一直迭代到k个特征

sfs = SFS(xgb.XGBClassifier(max_depth=5,
                        learning_rate=0.1,
                        n_estimators=500,
                        min_child_weight=0.5,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=10,
                        subsample=0.5,
                        colsample_bytree=0.8,
                        nthread=4,
                        scale_pos_weight=1),
# sfs = SFS(catboost.CatBoostRegressor(n_estimators=500,learning_rate=0.01),
          k_features=x.shape[1],
          forward=True,
          floating=False,
          verbose=2,
          scoring='precision',
          cv=3)  # cv表示交叉验证
sfs = sfs.fit(tran_x, tran_y)
# 逐步向前筛选结果，包括特征个数，最优特征组合及其r2
sfs_result = sfs.subsets_
print(sfs_result)

In [ ]:
df_sfs = pd.DataFrame(sfs_result)
# DataFrame转置
df_sfs_T=pd.DataFrame(df_sfs.values.T,index=df_sfs.columns,columns=df_sfs.index)
df_sfs_T=df_sfs_T.reset_index(drop=True)
# 保存逐步向前筛选结果

r2_list=list(df_sfs_T['avg_score'])
feature_list=list(df_sfs_T['feature_names'])

df_feature_test=pd.DataFrame({'特征':feature_list,'r2':r2_list})
df_feature_test=df_feature_test.reset_index(drop=True)
# 保存模型测试和测试结果到本地文件
writer = pd.ExcelWriter(project_path + '/data/result/df_2_逐步向前特征测试结果.xlsx')
df_feature_test.to_excel(writer)
writer.save()

In [ ]:
# 根据逐步向前测试结果筛选最优特征组合
r2_max=max(r2_list)
print(r2_max)
r2_max_index=r2_list.index(r2_max)
df_feature_select=df_feature_test.iloc[r2_max_index]
writer = pd.ExcelWriter(project_path + '/data/result/df_2_逐步向前特征选择结果.xlsx')
df_feature_select.to_excel(writer)
writer.save()

In [ ]:
feature_extract_list = list(feature_list[r2_max_index])
feature_extract_list.append('bmd_label')
df_extract = df_model[feature_extract_list]
writer = pd.ExcelWriter(project_path + '/data/result/df_2_逐步向前筛选后的建模数据.xlsx')
df_extract.to_excel(writer)
writer.save()

In [ ]:
# 逐步向前R2折线图
metrics=sfs.get_metric_dict()
# 截取部分字典数据，取30个特征生成逐步向前R2折线图
metrics_part={key:value for key,value in metrics.items() if key <30}
fig = plot_sfs(metrics_part, kind='std_err',color='r')
plt.title('Sequential Forward Selection (Precision)')
plt.grid()
# plt.show()
# 判断图片保存路径是否存在，否则创建
jpg_path = project_path + "/jpg"
mkdir(jpg_path)
plt.savefig(jpg_path + "/逐步向前特征选择R2折线图.jpg", dpi=300)
plt.clf()  # 删除前面所画的图